## Feature engineering

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('C:/Users/danie/Documents/GitHub/customer-loyalty-segmentation/data/processed/clean_data.csv')

In [12]:
df = df[df['Country'] == 'United Kingdom']
df

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom
4,489434,22064,PINK DOUGHNUT TRINKET POT,24,2009-12-01 07:45:00,1.65,13085.0,United Kingdom
...,...,...,...,...,...,...,...,...
352757,538171,22271,FELTCRAFT DOLL ROSIE,2,2010-12-09 20:01:00,2.95,17530.0,United Kingdom
352758,538171,22750,FELTCRAFT PRINCESS LOLA DOLL,1,2010-12-09 20:01:00,3.75,17530.0,United Kingdom
352759,538171,22751,FELTCRAFT PRINCESS OLIVIA DOLL,1,2010-12-09 20:01:00,3.75,17530.0,United Kingdom
352760,538171,20970,PINK FLORAL FELTCRAFT SHOULDER BAG,2,2010-12-09 20:01:00,3.75,17530.0,United Kingdom


#### Features based on RFM (Recency, Frequency, Monetary)

In [20]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

latest_date = df['InvoiceDate'].max()

rmf = df.groupby(['Customer ID']).agg({
    'InvoiceDate': lambda x: (latest_date - x.max()).days,
    'Invoice': 'count',
    'Price': 'sum'
    }).rename(columns={'InvoiceDate': 'Recency', 'Invoice': 'Frequency', 'Price': 'Monetary'})

rmf

,Recency,Frequency,Monetary
Customer ID,,,
12346.0,164,33,206.36
12608.0,39,12,22.40
12745.0,121,15,27.12
12746.0,175,17,59.10
12747.0,4,135,485.46
...,...,...,...
18283.0,17,211,417.22
18284.0,66,22,38.56
18285.0,295,8,24.20


#### Other purchase behavior metrics

In [25]:
df_agg = df.groupby(['Customer ID']).agg({
    'Quantity': 'sum',
    'Price': 'mean'
}).rename(columns={'Quantity': 'Total Quantity', 'Price': 'Average Price'})

df_customers = pd.merge(rmf, df_agg, on='Customer ID', how = 'left')

df_customers

,Recency,Frequency,Monetary,Total Quantity,Average Price
Customer ID,,,,,
12346.0,164,33,206.36,70,6.253333
12608.0,39,12,22.40,83,1.866667
12745.0,121,15,27.12,299,1.808000
12746.0,175,17,59.10,97,3.476471
12747.0,4,135,485.46,1210,3.596000
...,...,...,...,...,...
18283.0,17,211,417.22,315,1.977346
18284.0,66,22,38.56,275,1.752727
18285.0,295,8,24.20,58,3.025000


#### Features Normalization

Normalizate numeric features like rfm are important to garantee that the data is in the same scale, what's important to clusterization algorithms. Thats garantee that no variable will dominate the clusterization based on their scale. Monetary variable for example, can dominate the clusterization because monetary values are usually higher than recenty and frequency values.

In [27]:
from sklearn.preprocessing import StandardScaler

features_to_scale = ['Recency', 'Frequency', 'Monetary', 'Total Quantity', 'Average Price']

scaler = StandardScaler()
df_customers[features_to_scale] = scaler.fit_transform(df_customers[features_to_scale])

print(df_customers)

              Recency  Frequency  Monetary  Total Quantity  Average Price
Customer ID                                                              
12346.0      0.749138  -0.305504 -0.005997       -0.504126       4.146301
12608.0     -0.530946  -0.430607 -0.446982       -0.491478      -0.855291
12745.0      0.308789  -0.412735 -0.435667       -0.281330      -0.922182
12746.0      0.861785  -0.400820 -0.359005       -0.477858       0.980176
12747.0     -0.889369   0.302139  0.663055        0.604990       1.116461
...               ...        ...       ...             ...            ...
18283.0     -0.756241   0.754892  0.499472       -0.265764      -0.729097
18284.0     -0.254448  -0.371034 -0.408243       -0.304680      -0.985202
18285.0      2.090666  -0.454436 -0.442667       -0.515801       0.465418
18286.0      0.206382  -0.150615 -0.112599       -0.061453       0.144911
18287.0     -0.756241  -0.096999 -0.141318        0.168154      -0.470033

[3838 rows x 5 columns]
